In [2]:
import scanpy as sc
from scipy.io import mmwrite, mmread
import pandas as pd
import numpy as np

In [10]:
atac_adata = sc.read_h5ad('scButterfly/data/lymphoma/lymphoma_ATAC.h5ad')
rna_adata = sc.read_h5ad('scButterfly/data/lymphoma/lymphoma_RNA.h5ad')

/opt/conda/lib/python3.11/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [11]:
mmwrite('Polarbear/data/lymphoma_ATAC.mtx', atac_adata.X)
mmwrite('Polarbear/data/lymphoma_RNA.mtx', rna_adata.X)

In [59]:
np.savetxt('Polarbear/data/Lymphoma_ATAC_chr.csv', np.array(lym_RNA.var_names), delimiter=',', fmt='%s')

In [34]:
data_rna_batch = mmread('Polarbear/data/adultbrainfull50_rna_outer_snareseq_barcodes_dataset.mtx').tocsr()


In [40]:
a = data_rna_batch.toarray()

In [ ]:
chr_annot = pd.read_csv('Polarbear/data/Lymphoma_ATAC_chr.csv', sep=':', header=None)
chr_annot.columns = ['chr','pos']
chr_list = {}
for chri in chr_annot['chr'].unique():
    if chri not in ['chrX','chrY']:
        chr_list[int(chri[3:])] = [i for i, x in enumerate(chr_annot['chr']) if x == chri];

chr_list_range = []
for chri in chr_list.keys():
    chr_list_range += chr_list[chri]

In [60]:
data_rna_barcode = pd.read_csv('Polarbear/data/adultbrainfull50_rna_outer_snareseq_barcodes.tsv', delimiter='\t')
barcode_list = data_rna_barcode['index'].to_list()

['09K_ACCCTCCGAAGC',
 '09I_GAGGTTTGTAGA',
 '09C_ACGCATCTCTCG',
 '09J_CAGCGTGCTATC',
 '09K_TGATCCCATGAT',
 '09F_ATATGATGTACG',
 '09B_GCCAGCAACGGA',
 '09F_GCCCGACTCGCA',
 '09C_CCCCTAAGTGGG',
 '09K_CTGACGTGCTCA',
 '09K_GGACACAGTACG',
 '09E_TTAATGCGCAAG',
 '09C_TATTGTTCCAAC',
 '09B_CGGATGAACACA',
 '09E_ACCGTACCTAGA',
 '09I_GCGGACTGCCCG',
 '09K_GGGGTGCTTGTT',
 '09F_CATCCAAGAAAC',
 '09J_ACGAAGAAAGAT',
 '09K_GACCCTTGCTGA',
 '09F_TCGTCAGCCACG',
 '09I_AAGCTTCCGCTC',
 '09H_TGAGGTCCGGGA',
 '09I_GGTCCAGCTAAC',
 '09A_ATAGTCAGCGGT',
 '09H_CCCGTCAAAACT',
 '09B_CATTTTCTACCC',
 '09I_GCCTCACTGCAT',
 '09G_CACAAAGATTGC',
 '09B_AGAGCACTCGTC',
 '09E_TCAGTTTGTTAG',
 '09I_GCTCGAGTCTGA',
 '09A_AGCTCGGAATTA',
 '09E_TGACAGGGCACC',
 '09A_ATCGGACAGATA',
 '09J_TTCTATGAACAA',
 '09F_ACCTGTTAAGTG',
 '09D_GAGTCATCTTAA',
 '09G_TTCGACAAAAAT',
 '09G_TTCGAATGGGAA',
 '09D_AGCTTCAAAGGT',
 '09E_AGGTCCCTATTC',
 '09A_AGCCCGCTTCTG',
 '09A_CGTCGAGAGGGC',
 '09L_AGTTGGTAAGAG',
 '09J_GAATTAGCATAT',
 '09C_ATGAAGGAACAG',
 '09E_TAGTCGC

In [4]:
pd.DataFrame(list(lym_RNA.obs_names), columns=['index']).to_csv('Polarbear/data/lym_RNA_barcodes.csv', index=False)

In [8]:
data_rna_barcode = pd.read_csv('Polarbear/data/lym_RNA_barcodes.csv', sep=',', header=0)
barcode_list = data_rna_barcode['index'].to_list()